In [ ]:
import keras
from keras.layers import Conv2D, MaxPool2D, Input, Conv2DTranspose, BatchNormalization, Rescaling

In [ ]:
# Input layer
input_layer = Input(shape=(512,640,3), name='input_layer')
x = Rescaling(scale=1/255.0)(input_layer)

# Hidden layers
x = Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2,2))(x)

x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2,2))(x)

x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2,2))(x)

x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2,2))(x)

x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(filters=64, kernel_size=3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(filters=32, kernel_size=3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(filters=16, kernel_size=3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(filters=8, kernel_size=3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)

# Output layer
output_layer_heads = Conv2D(filters=1, kernel_size=1, activation='relu', kernel_initializer='he_uniform', name='heads_layer')(x)  #pointwise convolution
output_layer_centroids = Conv2D(filters=1, kernel_size=1, activation='relu', kernel_initializer='he_uniform', name='centroids_layer')(x)

# Defining the model
model = keras.models.Model(inputs=input_layer, outputs=[output_layer_heads, output_layer_centroids])

In [ ]:
# Compiling
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001), 
              loss={'heads_layer': 'mse',
                    'centroids_layer': 'mse'})

In [ ]:
# Fitting
no_epochs=12
batch_size=3 
history=model.fit(x=train_data, 
                  y={'heads_layer': gt_data_heads,
                     'centroids_layer': gt_data_centroids}, 
                  validation_data=(validation_data, {'heads_layer': gt_validation_heads,
                                                     'centroids_layer': gt_validation_centroids}),
                  batch_size=batch_size, 
                  epochs=no_epochs, 
                  shuffle=True)